# Lab - Game of Life on GPU - With Jupyter & Cling CUDA C++

The Game of Life is a system designed by the mathematician John Conway, based on a two-dimensional cellular automaton. The game area is divided into rows and columns. We assume a **square game area** in this lab. Each cell can occupy one of two states, which are called alive and dead. The next generation of a configuration evolves from simple rules.

* A dead cell with exactly three living neighbors is reborn in the next generation.
* A living cell with two or three living neighbours remains alive in the next generation.
* Living cells with a different number of living neighbours die in the next generation.

It is possible to define other rules. Conway's world is a 23/3 world. (One cell stays alive with 2 or 3 living neighbours. A cell is reborn with exactly 3 living neighbours.)

Further meaningful rules of the game and starting worlds can be read under [Wikipedia - Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life).

## Initialization

This adds required includes and libraries. Press __Shift + Enter__ to execute.

In [ ]:
#pragma cling(add_include_path "/usr/local/cuda/include")
#pragma cling(add_library_path "/usr/local/lib")

#include <iostream>

#pragma cling(load "cuda.so")
#pragma cling(load "libcudart.so")
#pragma cling(load "libnvrtc.so")
#pragma cling(load "libPNGwriter.so")

#include "task.hpp"

Define the dimension and number of iterations. If these numbers are changed again, a _Kernel -> Restart ..._ is required

In [ ]:
int dim = 10;
int iteration = 10;

Task1 t1(dim);
Task2 t2(dim);
Task3 t3(dim);

## 1. Task
<p>Implement the kernel that copies the shadow rows and columns.</p>
<img src="graphics/ghostcells.svg" width="50%"/>
<p>To do this, fill in the code skeleton in the following cell and then execute the following cell to check its implementation.</p>

In [ ]:
%%file copy_ghostcells.cu

__global__ void copy_ghostrows(int dim, int *world)
{
    int id = 0 /* TODO */;

    if (0 /* TODO */)
    {
        //Copy first real row to bottom ghost row
        world[0 /* TODO */] = world[0 /* TODO */];
        //Copy last real row to top ghost row
        world[0 /* TODO */] = world[0 /* TODO */];
    }
}

__global__ void copy_ghostcols(int dim, int *world)
{
    int id = 0 /* TODO */;

    if (0 /* TODO */)
    {
        //Copy first real column to right most ghost column
        world[0 /* TODO */] = world[0 /* TODO */];
        //Copy last real column to left most ghost column
        world[0 /* TODO */] = world[0 /* TODO */];
    }
}

Define the grid and block sizes.

In [ ]:
// using 1D grid for copy kernel
dim3 threads_1d(128);
dim3 blocks_1d((dim+2-1)/threads_1d.x+1);
// using 2D grid for update kernel
dim3 threads_2d(32, 4, 1);
dim3 blocks_2d((dim-1)/threads_2d.x+1, 
               (dim-1)/threads_2d.y+1, 
               1);

Set up the kernels and execute them. Mismatches will be shown in the output.

In [ ]:
t1.init_zero(); // set all values of the initial world to 0
t1.load_world("worlds/ghost_test.txt");
t1.init(); // delete all generated pictures and copy the source world to the GPU
// compile kernel file from above
t1.compile_kernel("copy_ghostcells.cu", "copy_ghostrows");
t1.compile_kernel("copy_ghostcells.cu", "copy_ghostcols");
// run kernels
t1.run_kernel("copy_ghostrows", blocks_1d, threads_1d);
t1.run_kernel("copy_ghostcols", blocks_1d, threads_1d);
t1.gen_diff(); // compare solution and result and generate a diff picture
t1.print_worlds(true); // print worlds to screen (clear output before)

1. Figure: expected solution
2. Figure: kernel solution
3. Figure: diff between expected and kernel solution (see legend)

<p> Legend: </p>
<p> <font> &#9608; </font> dead cell </p>
<p> <font color="ffffff">&#9608; </font> living cell </p>
<p> <font color="ff0000">&#9608; </font> should be a dead cell </p>
<p> <font color="00ff00">&#9608; </font> should be a living cell </p>

## 2. Task
Implement the actual Game-of-Life kernel. __Attention__, for the correct execution of the test cells it is necessary that task 1 has been solved correctly.

To verify your implementation, load sample worlds from the folder `worlds/` with the `load_world()` function.

In [ ]:
t2.compile_kernel("copy_ghostcells.cu", "copy_ghostrows");
t2.compile_kernel("copy_ghostcells.cu", "copy_ghostcols");
t2.init_zero();
t2.load_world("worlds/glider.txt"); // loading the world

`block.txt`, `beacon.txt` and `glider.txt` provide worlds for testing. The following table shows the expected result over several iterations:

| block.txt | beacon.txt | glider.txt |
|:--------- |:---------- |:---------- |
| <img src="graphics/game_of_life_block_with_border.svg"> | <img src="graphics/game_of_life_beacon.gif"> | <img src="graphics/game_of_life_animated_glider.gif">

As already in task 1, the code skeleton must be filled in first and then the following cell must be executed for verification.

There are two world buffers, one for reading the current state and one for writing (**could we also use a single buffer doing the update in-place?**).

In [ ]:
%%file GOL_GPU.cu

__global__ void GOL_GPU(int dim, int *world, int *newWorld) {
    int row = 0 /* TODO */;
    int col = 0 /* TODO */;
    int id  = 0 /* TODO */;

    int cell = world[id];

    int numNeighbors = world[0 /* TODO */]   // lower
      + world[0 /* TODO */]   // upper
      + world[0 /* TODO */]   // right
      + world[0 /* TODO */]   // left

      + world[0 /* TODO */]   // diagonal lower right
      + world[0 /* TODO */]   // diagonal upper left
      + world[0 /* TODO */]   // diagonal upper right
      + world[0 /* TODO */];  // diagonal lower left

    if (0 /* TODO */)
        newWorld[id] = 0;

    // 2) Any live cell with two or three live neighbours lives
    else if (0 /* TODO */)
        newWorld[id] = 1;

    // 3) Any live cell with more than three live neighbours dies
    else if (0 /* TODO */)
        newWorld[id] = 0;

    // 4) Any dead cell with exactly three live neighbours becomes a live cell
    else if (0 /* TODO */)
        newWorld[id] = 1;

    else
        newWorld[id] = cell;
}

In [ ]:
t2.init();
t2.compile_kernel("GOL_GPU.cu", "GOL_GPU");
for(int i = 0; i < iteration; ++i){
    // invokes the three kernels in the following order
    t2.run_kernel("copy_ghostrows",       // kernel-1 name
                  blocks_1d, threads_1d,  // launch config for kernel-1
                  "copy_ghostcols",       // kernel-2
                  blocks_1d, threads_1d,  // launch config-2
                  "GOL_GPU",              // kernel-3
                  blocks_2d, threads_2d); // launch config-3
}
t2.print_worlds(true);

**Note:** To speed up or slow down the animation, execute the following cell. The value indicates the pause time between two frames in milliseconds. Default is 800ms.

In [ ]:
t2.set_sleep_time(500)

**Note:** Task 3 actually wants a shared memory implementation, but you can refactor the kernel from above with a grid-stride loop version or with other rules, or worlds, and compare the output, or the runtimes in the subsequent benchmark. The dimension must be increased though, and no one tested this before. Feel free to play around :) In task.hpp you can edit the C++ code for these task objects if needed.

## 3. Task

Extend the game of life with shared memory to improve performance. First test your implementation as in task 2, then run the benchmark cells to measure and compare the times of Global Memory and Shared Memory implementation. 

In [ ]:
t3.compile_kernel("copy_ghostcells.cu", "copy_ghostrows");
t3.compile_kernel("copy_ghostcells.cu", "copy_ghostcols");
t3.init_zero();
t3.load_world("worlds/glider.txt");

In [ ]:
%%file GOL_SM_GPU.cu

// SM_size is (dim+2)*(dim+2)
template<int SM_size>
__global__ void GOL_SM_GPU(int dim, int *world, int *newWorld) {
   
    __shared__ int s_world[SM_size];

    int row = 0 /* TODO */;
    int col = 0 /* TODO */;
    int id  = 0 /* TODO */;
   
    s_world[id] = world[id];
    __syncthreads();

    if( 0 /* TODO */ ){
       
        int numNeighbors;
        int cell = world[id];

// same as in task 2

        numNeighbors =   s_world[0 /* TODO */]   // lower
          + s_world[0 /* TODO */]   // upper
          + s_world[0 /* TODO */]   // right
          + s_world[0 /* TODO */]   // left

          + s_world[0 /* TODO */]   // diagonal lower right
          + s_world[0 /* TODO */]   // diagonal upper left
          + s_world[0 /* TODO */]   // diagonal upper right
          + s_world[0 /* TODO */];  // diagonal lower left

        if (0 /* TODO */)
            newWorld[id] = 0;

        // 2) Any live cell with two or three live neighbours lives
        else if (0 /* TODO */)
            newWorld[id] = 1;

        // 3) Any live cell with more than three live neighbours dies
        else if (0 /* TODO */)
            newWorld[id] = 0;

        // 4) Any dead cell with exactly three live neighbours becomes a live cell
        else if (0 /* TODO */)
            newWorld[id] = 1;

        else
            newWorld[id] = cell;
    }
}

In [ ]:
t3.init();
t3.compile_kernel("GOL_SM_GPU.cu", "GOL_SM_GPU", (dim + 2) * (dim + 2));
for(int i = 0; i < iteration; ++i)
    t3.run_kernel("copy_ghostrows",       // kernel-1 name
                  blocks_1d, threads_1d,  // launch config for kernel-1
                  "copy_ghostcols",       // kernel-2
                  blocks_1d, threads_1d,  // launch config-2
                  "GOL_SM_GPU",           // kernel-3
                  blocks_2d, threads_2d); // launch config-3
t3.print_worlds(true);

### Benchmark

In [ ]:
// prepare two additional tasks for the benchmark

int bench_dim = 30;

// using 1D grid for copy kernel
dim3 threads_bench_1d(128);
dim3 blocks_bench_1d((bench_dim+2-1)/threads_bench_1d.x+1);
// using 2D grid for update kernel
dim3 threads_bench_2d(32,4,1);
dim3 blocks_bench_2d((bench_dim-1)/threads_bench_2d.x+1, (bench_dim-1)/threads_bench_2d.y+1, 1);

Task2 gm_kernel(bench_dim);
Task3 sm_kernel(bench_dim);

gm_kernel.compile_kernel("copy_ghostcells.cu", "copy_ghostrows");
gm_kernel.compile_kernel("copy_ghostcells.cu", "copy_ghostcols");
gm_kernel.init_zero();
gm_kernel.load_world("worlds/bench.txt");
gm_kernel.init();
gm_kernel.compile_kernel("GOL_GPU.cu", "GOL_GPU");

sm_kernel.compile_kernel("copy_ghostcells.cu", "copy_ghostrows");
sm_kernel.compile_kernel("copy_ghostcells.cu", "copy_ghostcols");
sm_kernel.init_zero();
sm_kernel.load_world("worlds/bench.txt");
sm_kernel.init();
// (file_path, kernel_name, total_shared_memory_size)
sm_kernel.compile_kernel("GOL_SM_GPU.cu", "GOL_SM_GPU", (bench_dim + 2) * (bench_dim + 2));

In [ ]:
int bench_iterations = 20000;

std::cout << "Total time of " << bench_iterations << " iterations" << std::endl;
std::cout << "Global Memory kernel: " << gm_kernel.bench_kernel(bench_iterations, // number_of_iterations,
                                                                "copy_ghostrows",
                                                                blocks_bench_1d, threads_bench_1d, // copy_ghostrows
                                                                "copy_ghostcols",
                                                                blocks_bench_1d, threads_bench_1d, // copy_ghostcols
                                                                "GOL_GPU",
                                                                blocks_bench_2d, threads_bench_2d) // GOL_GPU
          << "ms" << std::endl;
std::cout << "Shared Memory kernel: " << sm_kernel.bench_kernel(bench_iterations, 
                                                                "copy_ghostrows",
                                                                blocks_bench_1d, threads_bench_1d, // copy_ghostrows
                                                                "copy_ghostcols",
                                                                blocks_bench_1d, threads_bench_1d, // copy_ghostcols
                                                                "GOL_SM_GPU",
                                                                blocks_bench_2d, threads_bench_2d) // GOL_SM_GPU 
          << "ms" << std::endl;

## Bonus

With the following cell you can create your own world. `0` represents a dead cell and `X` is a living cell. You can also enlarge the world by the dim value in the first part of this notebook (the world must be square, kernel must be restarted, subsequent calls must be rerun).

To use that world below (execute first), change the path of the `load_world()` function in task 2 or 3 to `custom.txt`.

In [ ]:
%%file custom.txt
0 0 0 0 0 0 0 0 0 0
0 0 X 0 0 0 X X 0 0
0 0 0 X 0 0 X X 0 0
0 X X X 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 X X 0 0 0 0 0
0 0 0 X X 0 0 0 0 0
0 0 0 0 0 X X 0 0 0
0 0 0 0 0 X X 0 0 0
0 0 0 0 0 0 0 0 0 0